In [1]:
# 📌 YOLO 모델 사용을 위한 패키지 설치
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="gH4DnlLg4Gi5fnlGzCMa")
project = rf.workspace("industrialaicapstone").project("capstone_project-iwcq5")
version = project.version(9)
dataset = version.download("yolov5")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 132.3 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to capstone_project-9 in yolov5pytorch:: 100%|██████████| 3184/3184 [00:00<00:00, 5634.47it/s]


In [3]:
import os  # 파일 경로 처리를 위한 파이썬 모듈

# 각 데이터 경로 설정
train_path = '/content/capstone_project-9/train/images'
val_path = '/content/capstone_project-9/valid/images'
test_path = '/content/capstone_project-9/test/images'  # 상대경로 ../test/images → 절대경로로 변경

# 🔸 폴더 안의 .jpg, .png 같은 이미지 파일 수를 세는 함수 정의
def count_images(path):
    return len([f for f in os.listdir(path) if f.endswith(('.jpg', '.png', '.jpeg'))])

# 🔹 이미지 개수 출력
print(f"✅ Train 이미지 수: {count_images(train_path)}장")
print(f"✅ Validation 이미지 수: {count_images(val_path)}장")
print(f"✅ Test 이미지 수: {count_images(test_path)}장")


✅ Train 이미지 수: 1359장
✅ Validation 이미지 수: 132장
✅ Test 이미지 수: 95장


In [4]:
import os

# 클래스 이름 리스트
CLASS_NAMES = ['gown_on', 'hairnet_on', 'person']
class_counts = {i: 0 for i in range(len(CLASS_NAMES))}  # 클래스 번호별로 개수 초기화

# 모든 어노테이션(txt) 폴더 경로
label_paths = [
    '/content/capstone_project-9/train/labels',
    '/content/capstone_project-9/valid/labels',
    '/content/capstone_project-9/test/labels'
]

# 각 폴더 순회하면서 .txt 파일을 읽는다
for folder in label_paths:
    for file in os.listdir(folder):
        if file.endswith('.txt'):
            with open(os.path.join(folder, file), 'r') as f:
                for line in f:
                    class_id = int(line.strip().split()[0])  # 첫 숫자가 클래스 번호
                    if class_id in class_counts:
                        class_counts[class_id] += 1

# 결과 출력
print("✅ 클래스별 객체 수:")
for class_id, count in class_counts.items():
    print(f" - {CLASS_NAMES[class_id]}: {count}개")


✅ 클래스별 객체 수:
 - gown_on: 3273개
 - hairnet_on: 4018개
 - person: 4492개


In [5]:
import os
import random
import shutil
from glob import glob

# 🔹 현재 이미지/라벨 폴더 경로들
base_dir = '/content/capstone_project-9'
image_dirs = [
    os.path.join(base_dir, 'train/images'),
    os.path.join(base_dir, 'valid/images'),
    os.path.join(base_dir, 'test/images')
]
label_dirs = [
    os.path.join(base_dir, 'train/labels'),
    os.path.join(base_dir, 'valid/labels'),
    os.path.join(base_dir, 'test/labels')
]

# 🔹 모든 이미지/라벨 파일 경로 리스트로 수집 (.jpg/.txt)
all_images = []
all_labels = []

for img_dir, lbl_dir in zip(image_dirs, label_dirs):
    for img_path in glob(os.path.join(img_dir, '*.jpg')):
        filename = os.path.basename(img_path)
        label_path = os.path.join(lbl_dir, filename.replace('.jpg', '.txt'))
        if os.path.exists(label_path):  # 라벨이 있는 경우만
            all_images.append(img_path)
            all_labels.append(label_path)

# 🔹 이미지와 라벨 쌍 묶기
paired = list(zip(all_images, all_labels))
random.shuffle(paired)  # 무작위 셔플

# 🔹 총 수와 목표 수 계산
total = len(paired)
n_train = int(total * 0.75)
n_val = int(total * 0.15)
n_test = total - n_train - n_val

# 🔹 새로 나눌 데이터셋
splits = {
    'train': paired[:n_train],
    'valid': paired[n_train:n_train + n_val],
    'test': paired[n_train + n_val:]
}

# 🔹 새로 옮길 디렉토리 생성
for split in splits:
    os.makedirs(os.path.join(base_dir, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(base_dir, split, 'labels'), exist_ok=True)

# 🔹 파일 이동
for split, items in splits.items():
    for img_path, label_path in items:
        shutil.move(img_path, os.path.join(base_dir, split, 'images', os.path.basename(img_path)))
        shutil.move(label_path, os.path.join(base_dir, split, 'labels', os.path.basename(label_path)))

print("✅ 모든 데이터를 75:15:10 비율로 재분할 완료했습니다.")


✅ 모든 데이터를 75:15:10 비율로 재분할 완료했습니다.


In [6]:
import os

# ✅ 기준 경로
base_dir = '/content/capstone_project-9'

# ✅ 분할 대상 리스트
splits = ['train', 'valid', 'test']

# ✅ 각 폴더에서 이미지 수와 라벨 수를 출력
for split in splits:
    image_path = os.path.join(base_dir, split, 'images')
    label_path = os.path.join(base_dir, split, 'labels')

    image_count = len([f for f in os.listdir(image_path) if f.endswith('.jpg')])
    label_count = len([f for f in os.listdir(label_path) if f.endswith('.txt')])

    print(f"📂 {split.upper()} SET")
    print(f" - 이미지 수: {image_count}장")
    print(f" - 라벨 수:   {label_count}개\n")


📂 TRAIN SET
 - 이미지 수: 1189장
 - 라벨 수:   1189개

📂 VALID SET
 - 이미지 수: 237장
 - 라벨 수:   237개

📂 TEST SET
 - 이미지 수: 160장
 - 라벨 수:   160개



In [7]:
# YAML 파일로 저장할 내용을 문자열로 작성합니다.
# """ """ 는 여러 줄 문자열을 쓸 수 있게 해주는 파이썬 문법입니다.
yaml_content = """
train: /content/capstone_project-9/train/images     # 학습(training)에 사용할 이미지들이 있는 폴더 경로
val: /content/capstone_project-9/valid/images       # 검증(validation)에 사용할 이미지들이 있는 폴더 경로
test: /content/capstone_project-9/test/images       # 테스트(test)에 사용할 이미지들이 있는 폴더 경로

nc: 3                                               # nc는 클래스(class)의 수를 의미합니다. 여기서는 총 3개의 클래스를 사용합니다.
names: ['gown_on', 'hairnet_on', 'person']          # names는 각 클래스의 이름입니다. 리스트 형태로 작성합니다.
"""

# 이제 위에서 만든 yaml_content 내용을 파일로 저장하겠습니다.
# 'with open(...) as f'는 파일을 열고 작업한 후 자동으로 닫아주는 안전한 방법입니다.
# "w"는 write(쓰기 모드)를 의미합니다. 즉, 파일을 새로 만들거나 기존 내용을 덮어씁니다.
with open("/content/capstone_project-9/data.yaml", "w") as f:
    f.write(yaml_content)  # 파일 f에 위에서 만든 YAML 문자열을 씁니다.


In [8]:
from ultralytics import YOLO
from google.colab import drive, files
import shutil
import os

# ✅ 1단계: 설정 값 변수 정의
yolo_version = 5         # 5 또는 8 (YOLOv5 / YOLOv8)
model_type = 'n'         # 'n': nano / 's': small / 'm': medium 등
epochs = 50
batch = 16
imgsz = 640

# ✅ 2단계: 모델 파일명 및 결과 폴더명 생성
model_path = f'yolov{yolo_version}{model_type}.pt'
run_name = f'yolov{yolo_version}{model_type}_ep{epochs}_bs{batch}_img{imgsz}'

# ✅ 3단계: 모델 불러오기
model = YOLO(model_path)

# ✅ 4단계: 모델 학습
model.train(
    data='/content/capstone_project-9/data.yaml',
    epochs=epochs,
    imgsz=imgsz,
    batch=batch,
    name=run_name
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
PRO TIP 💡 Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 5.31M/5.31M [00:00<00:00, 65.1MB/s]


Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/capstone_project-9/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov5n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov5n_ep50_bs16_img640, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspecti

100%|██████████| 755k/755k [00:00<00:00, 14.5MB/s]

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1      1760  ultralytics.nn.modules.conv.Conv             [3, 16, 6, 2, 2]              
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      4800  ultralytics.nn.modules.block.C3              [32, 32, 1]                   
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     29184  ultralytics.nn.modules.block.C3              [64, 64, 2]                   
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  3    156928  ultralytics.nn.modules.block.C3              [128, 128, 3]                 
  7                  -1  1    295424  ultralytics

...

                   all        237       1636       0.93      0.881      0.937      0.599



50 epochs completed in 0.115 hours.
Optimizer stripped from runs/detect/yolov5n_ep50_bs16_img640/weights/last.pt, 5.3MB
Optimizer stripped from runs/detect/yolov5n_ep50_bs16_img640/weights/best.pt, 5.3MB

Validating runs/detect/yolov5n_ep50_bs16_img640/weights/best.pt...
Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv5n summary (fused): 84 layers, 2,503,529 parameters, 0 gradients, 7.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.09it/s]


                   all        237       1636      0.931      0.881      0.937      0.599
               gown_on        220        456      0.907       0.84      0.908      0.532
            hairnet_on        181        550       0.95      0.863      0.927      0.574
                person        237        630      0.934      0.941      0.977      0.692
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 6.3ms postprocess per image
Results saved to runs/detect/yolov5n_ep50_bs16_img640


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e2118239d50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [9]:

# ✅ 5단계: Google Drive 마운트
drive.mount('/content/drive')

# ✅ 6단계: 결과 폴더 zip 압축
zip_path = f"/content/{run_name}.zip"
result_folder_path = f"/content/runs/detect/{run_name}"
shutil.make_archive(f"/content/{run_name}", 'zip', result_folder_path)

# ✅ 7단계: 사용자 PC로 zip 다운로드
files.download(zip_path)

# ✅ 8단계: Google Drive에 저장할 위치 지정 및 폴더 생성
drive_base = '/content/drive/MyDrive/capstone/yolov' + str(yolo_version)
os.makedirs(drive_base, exist_ok=True)

# ✅ 9단계: zip 파일 Google Drive로 복사
shutil.copy(zip_path, drive_base)

# ✅ 10단계: 전체 결과 폴더 Google Drive로 복사
shutil.copytree(result_folder_path,
                os.path.join(drive_base, run_name),
                dirs_exist_ok=True)

print(f"✅ {run_name} 모델 결과가 Google Drive에 성공적으로 저장되었습니다.")


Mounted at /content/drive


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ yolov5n_ep50_bs16_img640 모델 결과가 Google Drive에 성공적으로 저장되었습니다.
